# Deploy MOSTLY AI SDK Algorithm Model Package from AWS Marketplace

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 



---



The MostlyAI Synthetic Data Generator is a powerful tool that enables you to create high-quality synthetic data while preserving the statistical properties and relationships of your original data. This synthetic data generation process ensures privacy compliance while maintaining data utility for various use cases including testing, development, and machine learning model training.


> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to MOSTLY AI SDK Algorithm. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> MOSTLY AI SDK Algorithm(Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
import base64
import json
import uuid
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
from urllib.parse import urlparse
import pandas as pd
from sagemaker.algorithm import AlgorithmEstimator
from sagemaker.inputs import TrainingInput
import time

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
role = get_execution_role()

sagemaker_session = sage.Session()

region = sagemaker_session.boto_region_name

bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")
bucket

'sagemaker-eu-central-1-438217566446'

In [3]:
# MOSTLY AI SDK Algorithm
algorithm_arn = f"arn:aws:sagemaker:{region}:446921602837:algorithm/mostlyai-sdk-sagemaker-cpu-87738f64e93b3f59aa191d36e4fb5ed0"


In [4]:

# Define the S3 URI to your training data
input_data_uri = "s3://testbucketuser-bucket/sagemaker-tests/train/"  

# Initialize the AlgorithmEstimator
estimator = AlgorithmEstimator(
    algorithm_arn=algorithm_arn,
    role=role,
    instance_count=1,
    instance_type="ml.c5.4xlarge",  # Or another instance type supported by your algorithm
    input_mode="File",
    sagemaker_session=sagemaker_session,
)

# Optional: set hyperparameters if your algorithm requires any
estimator.set_hyperparameters(**{
     "configFile": "generator.json",
     "generatorFile": "generator.zip"
 })

# Define the training input
train_input = TrainingInput(
    s3_data=input_data_uri,
    content_type="text/csv",  # Adjust to what your algorithm expects
)

# Start the training job
time_postfix = f"{int(time.time())}"
job_name = f"mostlyai-sdk-training-example-{time_postfix}"
estimator.fit({"train": train_input}, job_name=job_name)

model_name = f"mostlyai-sdk-model-example-{time_postfix}"

model = estimator.create_model(name=model_name)


INFO:sagemaker:Creating training-job with name: mostlyai-sdk-training-example-1744894979


2025-04-17 13:03:01 Starting - Starting the training job...
2025-04-17 13:03:16 Starting - Preparing the instances for training...
2025-04-17 13:03:51 Downloading - Downloading the training image......
2025-04-17 13:04:37 Training - Training image download completed. Training in progress..INFO:__main__:Loaded hyperparameters from /opt/ml/input/config/hyperparameters.json: {'configFile': 'generator.json', 'generatorFile': 'generator.zip'}
INFO:mostlyai.engine._common:running on Python (3.12.10)
Initializing Synthetic Data SDK 4.5.2 in LOCAL mode 🏠
Connected to /root/mostlyai with 30 GB RAM, 16 CPUs, 0 GPUs available
Loading configuration from: /opt/ml/input/data/train/generator.json
INFO:mostlyai.sdk._data.conversions:Container accessible: True
INFO:mostlyai.sdk._data.conversions:Container created: <mostlyai.sdk._data.file.base.LocalFileContainer object at 0x7f2d23d1f170>
INFO:httpx:HTTP Request: POST http://127.0.0.1/api/v2/connectors?testConnection=true "HTTP/1.1 200 OK"
Created conne

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Create an endpoint

In [5]:
# create a deployable model 
endpoint_name = f"mostlyai-sdk-endpoint-example-{time_postfix}"

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.c5.4xlarge",
    endpoint_name=endpoint_name
)


INFO:sagemaker:Creating model package with name: mostlyai-sdk-model-example-1744894979


.........

INFO:sagemaker:Creating model with name: mostlyai-sdk-model-example-1744894979


INFO:sagemaker:Creating endpoint-config with name mostlyai-sdk-endpoint-example-1744894979
INFO:sagemaker:Creating endpoint with name mostlyai-sdk-endpoint-example-1744894979


------!

Once endpoint has been created, you would be able to perform real-time inference.

<Add code snippet that shows the payload contents>

### B. Perform real-time inference

In [6]:
# Path to your JSON file
config_path = "inference_config.json"

# Read and parse the JSON
with open(config_path, "r") as f:
    inference_config = json.load(f)

# Print or use the loaded config
print(inference_config)

{'downloadFormat': 'csv', 'generatorFile': 'generator.zip', 'syntheticDataFile': 'players_seasons_synthetic.zip', 'syntheticDatasetConfig': {'name': 'players and seasons small', 'description': None, 'tables': [{'name': 'playerssmall', 'configuration': {'sampleSize': 10, 'sampleSeedConnectorId': None, 'sampleSeedDict': None, 'sampleSeedData': None, 'samplingTemperature': 1.0, 'samplingTopP': 1.0, 'rebalancing': None, 'imputation': None, 'fairness': None, 'enableDataReport': True}}, {'name': 'seasons_forplayerssmall', 'configuration': {'sampleSize': None, 'sampleSeedConnectorId': None, 'sampleSeedDict': None, 'sampleSeedData': None, 'samplingTemperature': 1, 'samplingTopP': 1, 'rebalancing': None, 'imputation': None, 'fairness': None, 'enableDataReport': True}}]}}


In [7]:
# Set appropriate content type — adjust if needed
predictor.content_type = "application/json"

# Convert the inference_config dict to JSON string
payload = json.dumps(inference_config)

# Make the prediction call
response_str = predictor.predict(payload)
response = json.loads(response_str)

# Print the response
print(response)

{'message': 'Transform/Inference completed successfully', 'result': {'playerssmall': [{'bats': 'R', 'birthDate': '1957-08-29T00:00:00.000', 'country': 'USA', 'deathDate': None, 'height': 77.0, 'id': 'mostly66-453d-4784-8338-2ba3b5a4f24c', 'nameFirst': 'Gene', 'nameLast': '_RARE_', 'throws': 'R', 'weight': 139.0}, {'bats': 'L', 'birthDate': '1955-01-08T00:00:00.000', 'country': 'USA', 'deathDate': '1998-01-27T00:00:00.000', 'height': 68.0, 'id': 'mostlya9-9153-4c3a-b365-14dbb157dab4', 'nameFirst': '_RARE_', 'nameLast': '_RARE_', 'throws': 'R', 'weight': 140.0}, {'bats': 'R', 'birthDate': None, 'country': 'USA', 'deathDate': None, 'height': 73.0, 'id': 'mostly05-e92a-49f1-9bec-5f346a7e5be3', 'nameFirst': 'Jason', 'nameLast': '_RARE_', 'throws': 'R', 'weight': 185.0}, {'bats': 'R', 'birthDate': '1942-07-08T00:00:00.000', 'country': 'USA', 'deathDate': '1997-08-11T00:00:00.000', 'height': 72.0, 'id': 'mostly72-f6ec-4012-9685-72523ecd3a5d', 'nameFirst': 'Joe', 'nameLast': '_RARE_', 'throws'

### C. Visualize output

In [8]:
def visualize_output(response):
    message = response.get("message")
    print(f"📌 Message: {message}")
    result = response.get("result", {})
    if not result:
        print("No 'result' key found or it's empty.")
    else:
        for key, records in result.items():
            print(f"📌 Table: {key}")
            try:
                df = pd.DataFrame(records)
                display(df)
            except Exception as e:
                print(f"Could not display table for '{key}': {e}")

# Visualize each item under 'result' as a separate table
visualize_output(response)

📌 Message: Transform/Inference completed successfully
📌 Table: playerssmall


,bats,birthDate,country,deathDate,height,id,nameFirst,nameLast,throws,weight
0,R,1957-08-29T00:00:00.000,USA,None,77.0,mostly66-453d-4784-8338-2ba3b5a4f24c,Gene,_RARE_,R,139.0
1,L,1955-01-08T00:00:00.000,USA,1998-01-27T00:00:00.000,68.0,mostlya9-9153-4c3a-b365-14dbb157dab4,_RARE_,_RARE_,R,140.0
2,R,None,USA,None,73.0,mostly05-e92a-49f1-9bec-5f346a7e5be3,Jason,_RARE_,R,185.0
3,R,1942-07-08T00:00:00.000,USA,1997-08-11T00:00:00.000,72.0,mostly72-f6ec-4012-9685-72523ecd3a5d,Joe,_RARE_,R,182.0
4,,1928-07-08T00:00:00.000,USA,2018-02-24T00:00:00.000,NaN,mostly6d-c7b3-4945-9be2-ca4a5707d8db,Josh,_RARE_,R,NaN
5,R,1921-11-08T00:00:00.000,USA,None,70.0,mostlydf-a356-4465-bc54-94bf84cfc5f7,John,_RARE_,R,222.0
6,R,1989-04-16T00:00:00.000,D.R.,None,74.0,mostly98-32f7-4de0-ba0d-a2bb4d32e675,_RARE_,_RARE_,L,190.0
7,L,1861-11-17T00:00:00.000,USA,None,73.0,mostly9e-21e7-455b-b7e3-5a276ff88765,_RARE_,_RARE_,R,190.0
8,R,1994-10-29T00:00:00.000,USA,1938-01-27T00:00:00.000,73.0,mostlybc-4c88-4cb3-95c7-4453c823facf,Jason,_RARE_,R,190.0
9,R,1953-04-06T00:00:00.000,USA,2001-02-19T00:00:00.000,74.0,mostly69-a187-4fb0-95dc-951e3a0678a5,_RARE_,_RARE_,R,185.0


📌 Table: seasons_forplayerssmall


,AB,BB,CS,G,H,HR,R,RBI,SB,SO,league,players_id,team,year
0,4,1,0.0,14,0,0,3,0,0,4,NL,mostlybc-4c88-4cb3-95c7-4453c823facf,ATL,1934
1,7,0,0.0,8,0,0,0,0,0,2,NL,mostlydf-a356-4465-bc54-94bf84cfc5f7,SFN,2002
2,20,2,0.0,16,7,1,2,2,0,2,AL,mostly9e-21e7-455b-b7e3-5a276ff88765,KCA,1969
3,0,0,0.0,3,0,0,0,0,0,0,AL,mostly05-e92a-49f1-9bec-5f346a7e5be3,CL4,1969
4,2,2,0.0,37,3,0,3,1,0,7,NL,mostly72-f6ec-4012-9685-72523ecd3a5d,DET,1946
5,0,0,0.0,34,0,0,0,0,0,0,AL,mostly98-32f7-4de0-ba0d-a2bb4d32e675,LAA,2010
6,338,6,0.0,7,13,1,1,7,2,19,AL,mostly69-a187-4fb0-95dc-951e3a0678a5,KCA,1935
7,368,5,0.0,8,53,1,5,6,3,29,NL,mostlya9-9153-4c3a-b365-14dbb157dab4,PHA,1902
8,3,0,0.0,3,100,0,0,0,0,2,NL,mostly66-453d-4784-8338-2ba3b5a4f24c,CHA,1977
9,9,0,NaN,1,0,0,0,0,0,0,AL,mostly6d-c7b3-4945-9be2-ca4a5707d8db,_RARE_,1926


### D. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [9]:
model.sagemaker_session.delete_endpoint_config(endpoint_name)

INFO:sagemaker:Deleting endpoint configuration with name: mostlyai-sdk-endpoint-example-1744894979


## 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [11]:
# Run the batch-transform job
batch_job_name = f"mostlyai-sdk-batchjob-example-{time_postfix}"

s3 = boto3.client("s3")
output_s3 = f"batch-output/{time_postfix}/"
s3.put_object(Bucket=bucket, Key=output_s3)

transformer = model.transformer(1, "ml.c5.4xlarge", output_path=f"s3://{bucket}/{output_s3}")
transformer.transform("s3://testbucketuser-bucket/sagemaker-tests/inference/", content_type="application/json",job_name=batch_job_name)
transformer.wait()

INFO:sagemaker:Creating model with name: mostlyai-sdk-model-example-1744894979
INFO:sagemaker:Creating transform job with name: mostlyai-sdk-batchjob-example-1744894979


..............................INFO:__main__:=== Environment Variables ===
INFO:__main__:TRANSFORM_JOB_ARN: arn:aws:sagemaker:eu-central-1:438217566446:transform-job/mostlyai-sdk-batchjob-example-1744894979
INFO:__main__:PYTHON_SHA256: 07ab697474595e06f06647417d3c7fa97ded07afc1a7e4454c5639919b46eaea
INFO:__main__:SAGEMAKER_SAFE_PORT_RANGE: 10000-10999
INFO:__main__:PYTHONUNBUFFERED: TRUE
INFO:__main__:AWS_CONTAINER_CREDENTIALS_RELATIVE_URI: /v2/credentials/nnLEYNXXjWMxKrMTtoS8RvpOAQMhDhI7I6GNck1Blvk
INFO:__main__:HOSTNAME: 41235ce5b63c
INFO:__main__:PYTHON_VERSION: 3.12.10
INFO:__main__:HF_DATASETS_CACHE: /root/.cache/huggingface/datasets
INFO:__main__:AWS_REGION: eu-central-1
INFO:__main__:PWD: /opt/ml/code
INFO:__main__:TZ: UTC
INFO:__main__:TRANSFORMERS_CACHE: /root/.cache/huggingface/hub
INFO:__main__:HOME: /root
INFO:__main__:LANG: C.UTF-8
INFO:__main__:GPG_KEY: 7169605F62C751356D054A26A821E680E5FA6305
INFO:__main__:HF_HOME: /root/.cache/huggingface
INFO:__main__:SAGEMAKER_BIND_TO_

In [12]:
# output is available on following path
output_path = transformer.output_path

# Parse the S3 path
parsed = urlparse(output_path)
bucket_name = parsed.netloc
key = parsed.path.lstrip("/")

# Download the file content
s3 = boto3.client("s3")
response_obj = s3.get_object(Bucket=bucket_name, Key=key + "inference_config.json.out")
response_body = response_obj["Body"].read().decode("utf-8")

# Parse JSON content
batch_response = json.loads(response_body)

visualize_output(batch_response)

📌 Message: Transform/Inference completed successfully
📌 Table: playerssmall


,bats,birthDate,country,deathDate,height,id,nameFirst,nameLast,throws,weight
0,L,1858-02-15T00:00:00.000,USA,1964-05-08T00:00:00.000,68,mostlya2-ec9b-4f34-938e-6bc0d2b32aff,Tom,_RARE_,L,160
1,,1958-07-25T00:00:00.000,Venezuela,1956-02-08T00:00:00.000,67,mostly78-ff29-4348-9b1c-4f5dfc07f342,Don,_RARE_,,154
2,R,1894-12-15T00:00:00.000,USA,None,74,mostlydd-8b0a-4e26-a7c5-b585ad0af1a9,Rick,Thomas,R,200
3,B,1902-10-18T00:00:00.000,_RARE_,1995-06-16T00:00:00.000,77,mostlyac-50da-4787-999f-555b07b860fe,_RARE_,_RARE_,R,197
4,R,1892-12-24T00:00:00.000,USA,1918-08-08T00:00:00.000,72,mostly2a-4b06-41a0-8112-d7b038cc3951,_RARE_,_RARE_,R,170
5,B,1976-03-12T00:00:00.000,Venezuela,None,74,mostlyfc-36b1-4139-a971-2b551f242fc1,Matt,_RARE_,R,185
6,L,1967-07-21T00:00:00.000,D.R.,None,77,mostly2d-9d45-4bb0-a371-48a3901c47db,John,_RARE_,L,205
7,,1868-08-22T00:00:00.000,USA,1980-02-11T00:00:00.000,65,mostly4a-6af9-4d7b-b905-00841a579853,Tony,_RARE_,R,150
8,R,1945-10-04T00:00:00.000,USA,None,68,mostly06-f0f2-46df-bb13-025e1d2a54ad,_RARE_,_RARE_,R,160
9,L,1883-04-24T00:00:00.000,USA,None,72,mostly00-b631-4a6e-8c3c-6430e3056cd8,Matt,_RARE_,R,185


📌 Table: seasons_forplayerssmall


,AB,BB,CS,G,H,HR,R,RBI,SB,SO,league,players_id,team,year
0,62,0,0.0,119,11,0,1,1,2,17,NL,mostlyfc-36b1-4139-a971-2b551f242fc1,KCA,2002
1,3,1,NaN,1,2,0,1,0,0,1,NL,mostlya2-ec9b-4f34-938e-6bc0d2b32aff,SLN,1890
2,0,1,NaN,6,0,0,0,0,0,0,NL,mostly4a-6af9-4d7b-b905-00841a579853,FW1,1886
3,15,1,NaN,17,9,0,2,3,0,3,NL,mostlyac-50da-4787-999f-555b07b860fe,SLN,1953
4,26,5,1.0,12,17,3,13,7,3,4,AL,mostly00-b631-4a6e-8c3c-6430e3056cd8,TEX,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,248,47,3.0,17,65,12,35,8,4,69,AL,mostlyac-50da-4787-999f-555b07b860fe,DET,2002
60,3,0,0.0,39,0,0,0,0,0,0,NL,mostly2d-9d45-4bb0-a371-48a3901c47db,CIN,1993
61,1,0,0.0,67,0,0,0,0,0,0,NL,mostly2d-9d45-4bb0-a371-48a3901c47db,SFN,1998
62,0,0,0.0,74,0,0,0,0,0,0,NL,mostly2d-9d45-4bb0-a371-48a3901c47db,SLN,1995


## 4. Clean-up

### A. Delete the endpoint and the model

In [ ]:
sagemaker_session.delete_endpoint(endpoint_name)
model.delete_model()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

